# In the name of ALLAH

# Algebraic Attack on BiviumA
H. Hadipour Jan 23 2017, 3 Bahman 1396

In [7]:
from sage.crypto.util import *
from sage.crypto.boolean_function import BooleanFunction
from sage.sat.boolean_polynomials import solve as solve_sat
from sage.rings.polynomial.multi_polynomial_sequence import PolynomialSequence

# BiviumA Algorithm 

In [8]:
feedback1 = lambda a24, b15, b0, b1, b2: a24 + b15 + b0 + b1*b2
feedback2 = lambda b6, a27, a0, a1, a2: b6 + a27 + a0 + a1*a2
f = lambda b0, b15: b0 + b15
def BiviumA(N):
    P = GF(2)
    n1 = 93; n2 = 84         
    # Initialization
    A = [0] * (N + n1)
    B = [0] * (N + n2)
    Z = [0] * (N)
    # Random key generation
    A[13:n1] = [P.random_element() for r in xrange(80)]
    # Random IV generation
    B[4:n2] = [P.random_element() for r in xrange(80)]
    # Keystream calculation
    for i in xrange(N):    
        Z[i] = P(f(B[i], B[i + 15]))    
        A[i + n1] = P(feedback1(A[i + 24], B[i + 15], B[i], B[i + 1], B[i + 2]))
        B[i + n2] = P(feedback2(B[i + 6], A[i + 27], A[i], A[i + 1], A[i + 2]))
    keystream = Z
    Key = A[92:12:-1]
    IV = B[84:3:-1]
    return Key, IV, keystream

In [9]:
K, IV, KSS = BiviumA(313)

In [15]:
print('%x'%int(''.join([str(e) for e in KSS]),2))

312c734249d2fd53acfbd75062afb5d1050b7ce33ccb056435b3636532d5539cacbc0554292437


In [16]:
str_feedback1 = lambda a24, b15, b0, b1, b2: a24 + ' + ' +  b15 + ' + ' + b0 + ' + ' + b1 + '*' + b2
str_feedback2 = lambda b6, a27, a0, a1, a2: b6 + ' + ' + a27 + ' + ' + a0 + ' + ' + a1 + '*' + a2
str_f = lambda b0, b15: b0 + ' + ' + b15

# Extraction algebraic equations of BiviumA

In [17]:
def BiviumA_Equations(N, known_kss_msk):
    assert(len(known_kss_msk) <= N)
    nbitn = ceil(float(log(N, 2)))   
    K, IV, KS = BiviumA(N)
    n1 = 93; n2 = 84
    A = ['a' + str(i) for i in range(n1 + N)]
    B = ['b' + str(i) for i in range(n2 + N)]
    Z = ['z' + bin(i)[2:].zfill(nbitn) for i in range(N)]
    total_vars = A + B + Z        
    #Extraction equations in string format:
    Eqs = []
    for i in xrange(N):
        Eqs += [A[i + n1] + ' + ' + str_feedback1(A[i + 24], B[i + 15], B[i], B[i + 1], B[i + 2])]
        Eqs += [B[i + n2] + ' + ' + str_feedback2(B[i + 6], A[i + 27], A[i], A[i + 1], A[i + 2])]
        Eqs += [str_f(B[i], B[i + 15]) + ' + ' + Z[i]]
    #Substituaiton known values:
    known_variables = [Z[i] for i in known_kss_msk]
    known_values = [KS[i] for i in known_kss_msk]
    subs_dct = zip(known_variables, known_values)    
    for k in subs_dct:
        total_vars.remove(k[0])
        for i in range(len(Eqs)):
            Eqs[i] = Eqs[i].replace(k[0], str(k[1]))      
    #Conversion string equations to ring objects
    Ring = BooleanPolynomialRing(len(total_vars), total_vars)
    var_str = list(Ring.variable_names())
    n_vars = len(var_str)
    var_ring = [Ring.variable(i) for i in range(n_vars)]
    dct = dict(zip(var_str, var_ring))
    ring_evaluator = lambda eq: sage_eval(eq, locals = dct)    
    print(Eqs[0])
    Eqs = map(ring_evaluator, Eqs) 
    Eqs = PolynomialSequence(Eqs) 
    ########
    return K, IV, KS, Eqs

In [18]:
N = 177
known_vals = range(0, N, 1)
%time K, IV, KS, Eqs = BiviumA_Equations(N, known_vals)
print(Eqs)

a93 + a24 + b15 + b0 + b1*b2
CPU times: user 627 ms, sys: 21 ms, total: 647 ms
Wall time: 618 ms
Polynomial Sequence with 531 Polynomials in 531 Variables


# Solving by SAT

In [19]:
number_of_sols = 100
%time sols = solve_sat(Eqs, n = number_of_sols, s_verbosity = 8)
print('number of soluiton: %s' % len(sols))
for i in range(len(sols)):
    dl = sols[i]
    temp = [value for (key, value) in sorted(dl.items(), reverse=True)]
    if temp[92:12:-1] == K:
        print('The solution matches with the original key!')
        break

CPU times: user 41 s, sys: 2.53 s, total: 43.5 s
Wall time: 46.3 s
number of soluiton: 2
The solution matches with the original key!
